# Purpose: 

The purpose of this notebook is to utilise the files created from the first notebook, apply the rules for the creation of xlsx files for manual annotation

In [2]:
!pip install contractions
!pip install nltk 
!pip install spacy
!pip install spacy-language-detection
!pip install fasttext
!pip install langdetect
!pip install p_tqdm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 23.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 48.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for p_tqdm: filename=p_tqdm-1.4.0-py3-none-any.whl size=5383 sha256=9b09dff9affe48fe3d567e585eec4c72823b6dcac3889af78ff765cf746e91a1
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a2/54/28/97311859e49a55634542c9c839e4341fa00117ef1ae7ad0ffe
Successfully built p_tqdm


In [3]:
import pandas as pd 
import re
import os
import string
import contractions
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
import nltk.data
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import words
import re
import spacy
from spacy_language_detection import LanguageDetector
from spacy.language import Language
import fasttext
from langdetect import detect
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import numpy as np
from p_tqdm import p_imap
from pandas import DataFrame

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')
nltk.download('omw-1.4')


tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer.sent_end_chars=(':','.',',',';')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS)) - set(['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'elven', 'twelve' ])


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data] Downloading package words to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ec2-user/nltk_data...


In [36]:
def clean_contractions(text: str):
    """
    Clean contractions in the given text.

    Args:
        text (str): The input text to clean contractions.

    Returns:
        str: The text with contractions expanded and cleaned.
    """
    text = contractions.fix(text).lower()
    text = re.sub('can t', ' cannot ', text).strip()
    text = re.sub(' ll ', ' will ', text).strip()
    text = re.sub(' +', ' ', text).strip()
    text = re.sub(r' [^\w\s] ', '', text)
    return text


In [46]:
# text_cleaning - utility functions: text_cleaning.py
def basic_cleaning(text:string, punctuation:bool) -> str:
    """
    This function does some basic data cleaning.
    """
    for a_sign in ['\\n', '\\t', '☐', '☒', '\xa0', '●', '“', '”']:
        text = text.replace(a_sign," ")

    if punctuation:
        for a_punc in string.punctuation:
            text = text.replace(a_punc, " ")
    else:
        text =  re.sub(', ', ' ', text).strip()
    
    text = re.sub('\s+'," ", text).lower()
    return text.strip() 

def split_into_sentences(text:str) -> list:
    
    # Handle period (.)
    text = re.sub(r'([a-zA-Z)])\.([a-zA-Z-])', r'\1. \2', text)
    text = re.sub(r"([\d)])\.([a-zA-Z-])", r"\1. \2", text)
    text = re.sub(r"([a-zA-Z)])\.(\d)", r"\1. \2", text)
    #.Handle exclamation mark (!)
    text = re.sub(r'([a-zA-Z)])\!([a-zA-Z-])', r'\1! \2', text)
    text = re.sub(r"([\d)])\!([a-zA-Z-])", r"\1! \2", text)
    text = re.sub(r"([a-zA-Z)])\!(\d)", r"\1! \2", text)
    #? # Handle question mark (?)
    text = re.sub(r'([a-zA-Z)])\?([a-zA-Z-])', r'\1? \2', text)
    text = re.sub(r"([\d)])\?([a-zA-Z-])", r"\1? \2", text)
    text = re.sub(r"([a-zA-Z)])\?(\d)", r"\1? \2", text)
    #:  # Handle colon (:)
    text = re.sub(r'([a-zA-Z)])\:([a-zA-Z-])', r'\1: \2', text)
    text = re.sub(r"([\d)])\:([a-zA-Z-])", r"\1: \2", text)
    text = re.sub(r"([a-zA-Z)])\:(\d)", r"\1: \2", text)
    #;  # Handle colon (:)
    text = re.sub(r'([a-zA-Z)])\;([a-zA-Z-])', r'\1; \2', text)
    text = re.sub(r"([\d)])\;([a-zA-Z-])", r"\1; \2", text)
    text = re.sub(r"([a-zA-Z)])\;(\d)", r"\1; \2", text)
    return tokenizer.tokenize(text)

def remove_special_characters(text:str):
    """
    Remove special characters from the given text.

    Args:
        text (str): The input text from which special characters will be removed.

    Returns:
        str: The text with special characters removed.
    """
    
    
    # Replace "%" with "percent"
    text = re.sub(r"%", ' percent ', text)
    # Remove special characters except alphanumeric characters, spaces, and certain punctuation marks
    return re.sub(r"[^A-Za-z0-9\s!.,?]+", '', text)


def remove_stop_words(text:string):
    """
    Remove stop words from the given text.

    Args:
        text (str): The input text from which stop words will be removed.

    Returns:
        str: The text with stop words removed.
    """
    text = ' '.join([word for word in text.split() if word not in stop])
    return text 

def clean_contractions(text: str):
    """
    Clean contractions in the given text.

    Args:
        text (str): The input text to clean contractions.

    Returns:
        str: The text with contractions expanded and cleaned.
    """
    text = contractions.fix(text).lower()
    text = re.sub('can t', ' cannot ', text).strip()
    text = re.sub(' ll ', ' will ', text).strip()
    text = re.sub(' +', ' ', text).strip()
    text = re.sub(r' [^\w\s] ', '', text)
    return text


def removing_numbers(text: str):
    """
    Remove numbers and certain characters from the given text.

    Args:
        text (str): The input text from which numbers and characters will be removed.

    Returns:
        str: The text with numbers and characters removed.
    """
    text = re.sub('(?<=\d),(?=\s)', '', text)
    text = re.sub('(?<=\d).(?=\d)', '', text)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    # text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', text)
    text = re.sub(' +', ' ', text).strip()
    return text


def removing_non_english(text: str):
    """
    Remove non-English words from the given text.

    Args:
        text (str): The input text from which non-English words will be removed.

    Returns:
        str: The text with non-English words removed.
    """
    text = ' '.join([word for word in text.split() if word in words.words()])
    return text


def lammatize_text(text: str):
    """
    Lemmatize the words in the given text.

    Args:
        text (str): The input text to lemmatize.

    Returns:
        str: The text with lemmatized words.
    """
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text


def stem_text(text: str):
    """
    Perform stemming on the words in the given text.

    Args:
        text (str): The input text to perform stemming.

    Returns:
        str: The text with stemmed words.
    """
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text


def replace_words(text: str, word_list: dict):
    """
    Replace specified words in the text with corresponding replacements.

    Args:
        text (str): The input text in which words will be replaced.
        word_list (dict): A dictionary mapping words to their replacements.

    Returns:
        str: The text with replaced words.
    """
    for word in list(word_list):
        text = text.replace(word, word_list[word])
    return text


def text_cleaner(text: str, punctuation: bool, special_char: bool, stop_words: bool, contractions: bool,
                 numbers: bool, non_english: bool, lammatize: bool, stem: bool, replacements: dict):
    """
    Clean and preprocess the given text based on the specified cleaning options.

    Args:
        text (str): The input text to be cleaned.
        punctuation (bool): Whether to remove punctuation.
        special_char (bool): Whether to remove special characters.
        stop_words (bool): Whether to remove stop words.
        contractions (bool): Whether to expand and clean contractions.
        numbers (bool): Whether to remove numbers and certain characters.
        non_english (bool): Whether to remove non-English words.
        lemmatize (bool): Whether to lemmatize words.
        stem (bool): Whether to perform stemming.
        replacements (dict): A dictionary mapping words to their replacements.

    Returns:
        str: The cleaned text.
    """
    text = basic_cleaning(text, punctuation)
    if replacements is not None:
        text = replace_words(text, replacements)
    if special_char:
        text = remove_special_characters(text)
    if contractions:
        text = clean_contractions(text)
    if numbers:
        text = removing_numbers(text)
    if non_english:
        text = removing_non_english(text)
    if lammatize:
        text = lammatize_text(text)
    if stem:
        text = stem_text(text)
    if stop_words:
        text = remove_stop_words(text)
    return text



def creating_bigrams(text: str):
    """
    Create bigrams from the given text.

    Args:
        text (str): The input text to create bigrams from.

    Returns:
        list: A list of bigrams.
    """
    sentences = tokenizer.tokenize(text)
    bigrams = []
    for sentence in sentences:
        bigrams.extend(list(nltk.bigrams(sentence.split())))
    return bigrams


def creating_unigrams(text: str):
    """
    Create unigrams from the given text.

    Args:
        text (str): The input text to create unigrams from.

    Returns:
        list: A list of unigrams.
    """
    return text.split()



#### Detect Language


class languate_detection_fasttext():
    """
    Initialize the language detection model using FastText.

    This class detects the language of the text using FastText's language detection model.
    """

    def __init__(self):
        pretrained_lang_model = os.path.join('/home/ec2-user/SageMaker/amazon_thesis_detectron/utils', "lid.176.bin")
        self.model = fasttext.load_model(pretrained_lang_model)

    def detect_language(self, text):
        """
        Detect the language of the given text.

        Args:
            text (str): The input text to detect the language from.

        Returns:
            tuple: A tuple containing the detected language and its confidence score.
        """
        predictions = self.model.predict(text, k=1)  # returns top 2 matching languages
        return predictions[0][0].replace('__label__', ''), predictions[1][0]





In [47]:
def clean_reviews_rating_references(amazon_reviews:DataFrame, max_reviews = None) -> DataFrame:
    '''
    This function takes a DataFrame containing reviews, splits the review in sentences, looks for references to rating or reviews, and returns a new DataFrame.
    If desired, maximum number of reviews can be set. 
    '''
    amazon_reviews = amazon_reviews[(amazon_reviews["language"] == 'en') ].sample(frac  =1,  random_state=1).to_dict(orient = "records")
    if max_reviews:
        print(f"Process only {max_reviews} of {len(amazon_reviews)}")
        amazon_reviews = amazon_reviews[0:max_reviews]
    else:
        print(f"Process all {len(amazon_reviews)}.")

    def processing_reviews(chunk:list):
        amazon_review_cleaned = []
        for review in chunk:
            senteces = split_into_sentences(text = review['review_body'])
            for sentence in senteces:
                temp = {"asin": review["product_id"], 'review_id': review["review_id"], 'reviewer_id': review["account_link"], 'date': review["review_date"], "full_review": review["review_body"] }
                temp['sentence'] =  sentence
                temp['text_clean'] = text_cleaner(text = sentence, non_english= False,  punctuation= True, special_char= True, stop_words= True, contractions= True, numbers= False, lammatize= True, stem = True, replacements= None)
                temp['text_unigrams'] = creating_unigrams(temp['text_clean'])
                temp['rating_reference'] =  1 if ("rate" in temp['text_unigrams']) else 0
                temp['star_reference'] =  1 if ("star" in temp['text_unigrams']) else 0
                temp['review_reference'] =  1 if ("review" in temp['text_unigrams']) else 0
                amazon_review_cleaned.append(temp)
                del temp
        return amazon_review_cleaned

    print("Start multi-processing!")
    chunk_size = 100
    chunks = [amazon_reviews[i:i + chunk_size] for i in range(0, len(amazon_reviews), chunk_size)]
    print(len(chunks))
    multi_processing_output = [ x for x in p_imap(processing_reviews, chunks )]
    print(len(multi_processing_output))

    return DataFrame([item for sublist in multi_processing_output  for item in sublist]), multi_processing_output

In [48]:
language_detection = languate_detection_fasttext()
# language_detection.detect_language('hello my name is kennedy')[0]

In [49]:
# path to the processed files which have been processed from raw data json files
path_to_processed_raw_file = '/home/ec2-user/SageMaker/amazon_thesis_detectron/000_Create_Manual_Annotation_Files/001_office_data_processed'
os.listdir(path_to_processed_raw_file) # listing all the json files which have been converted to dataframes

['amazon_reviews_office_3.json.json',
 'amazon_reviews_office_2.json.json',
 'amazon_reviews_office_8.json.json',
 'amazon_reviews_office_0.json.json',
 'amazon_reviews_office_6.json.json',
 'amazon_reviews_office_9.json.json',
 'amazon_reviews_office_7.json.json',
 'amazon_reviews_office_4.json.json',
 'amazon_reviews_office_5.json.json',
 'amazon_reviews_office_1.json.json']

In [52]:
# path to save the files for manual tagging 
saved_folder_path= '/home/ec2-user/SageMaker/amazon_thesis_detectron/000_Create_Manual_Annotation_Files/office_tagged_data'

In [51]:
for file in os.listdir(path_to_processed_raw_file):
    tqdm.pandas()
        # Find the position of the first full stop
    first_full_stop_index = file.find('.')

    # Extract the substring before the first full stop
    extracted_text = file[:first_full_stop_index]
    # convert this to a function once check is complete: 
    amazon_reviews_0_df = pd.read_json(os.path.join(path_to_processed_raw_file, file)) # 1000 products 
    amazon_reviews_0_df.review_body = amazon_reviews_0_df.review_body.apply(lambda x : x.replace("\n", " ").strip())
    amazon_reviews_0_df.review_body = amazon_reviews_0_df.review_body.replace("\s+", " ")
    amazon_reviews_0_df.rating = amazon_reviews_0_df.rating.apply(lambda x: float(x.replace(" out of 5 stars", "")))
    amazon_reviews_0_df['language']= amazon_reviews_0_df.review_body.progress_apply(lambda x: language_detection.detect_language(x)[0])
    amazon_reviews = amazon_reviews_0_df.copy()
    amazon_reviews= amazon_reviews[['review_id', 'account_link', 'review_date', 'rating',
           'review_title', 'review_body', 'purchase_type', 'helpful_votes',
           'product_id', 'overall_rating', 'purchase_type', 'review_body',
           'language']]
    amazon_reviews_df, multi_processing_output = clean_reviews_rating_references(amazon_reviews=amazon_reviews[amazon_reviews['rating'] <= 2 ])
    amazon_reviews_df_temp = amazon_reviews_df[(amazon_reviews_df['rating_reference'] == 1) | (amazon_reviews_df['star_reference'] == 1) | (amazon_reviews_df['review_reference'] == 1)].reset_index(drop = True) 
    amazon_reviews_df_temp["sentence"] = amazon_reviews_df_temp["sentence"].apply(lambda x: ' ' + x)
    amazon_reviews_df_temp.to_excel(os.path.join(saved_folder_path,f'{extracted_text}.xlsx'))

100%|██████████| 187344/187344 [00:04<00:00, 40321.42it/s]
/tmp/ipykernel_8212/904060701.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  amazon_reviews = amazon_reviews[(amazon_reviews["language"] == 'en') ].sample(frac  =1,  random_state=1).to_dict(orient = "records")


Process all 25961.
Start multi-processing!
260


  0%|          | 0/260 [00:00<?, ?it/s]

260


100%|██████████| 179088/179088 [00:05<00:00, 35259.36it/s]


...

250


100%|██████████| 189493/189493 [00:04<00:00, 40639.53it/s]
/tmp/ipykernel_8212/904060701.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  amazon_reviews = amazon_reviews[(amazon_reviews["language"] == 'en') ].sample(frac  =1,  random_state=1).to_dict(orient = "records")


Process all 24616.
Start multi-processing!
247


  0%|          | 0/247 [00:00<?, ?it/s]

247


In [1]:
print("End of Notebook")

End of Notebook
